In [2]:

def run_forcast():
    import pandas as pd
    import numpy as np
    from statsmodels.tsa.statespace.sarimax import SARIMAX
    import warnings
    warnings.filterwarnings("ignore")
    df = pd.read_csv("pengeluaran_harian_2024_final.csv")
    df['date'] = pd.to_datetime(df['date'])

    monthly_data = df.groupby([df['date'].dt.to_period('M'), 'category'])['amount'].sum().reset_index()
    monthly_data['date'] = monthly_data['date'].dt.to_timestamp()
    monthly_data.rename(columns={'date': 'ds', 'amount': 'y'}, inplace=True)

    categories = monthly_data['category'].unique()
    forecast_results = {}

    order = (1, 1, 1)
    seasonal_order = (1, 1, 1, 12)

    for cat in categories:
        cat_data = monthly_data[monthly_data['category'] == cat].set_index('ds')['y']
        model = SARIMAX(cat_data, order=order, seasonal_order=seasonal_order,
                        enforce_stationarity=False, enforce_invertibility=False)
        results = model.fit(disp=False)
        steps = 2
        forecast = results.forecast(steps)
        forecast_results[cat] = forecast.values[0]

    forecast_date = cat_data.index[-1] + pd.DateOffset(months=1)
    forecast_df = pd.DataFrame(forecast_results, index=[forecast_date]).round(0)
    forecast_df = forecast_df.applymap(lambda x: f"Rp. {int(x):,}".replace(",", "."))
    
    print("One-Month Forecast:")
    print(forecast_df)

   
    

In [ ]:

def save_result_to_file(result, filename="forecast.txt"):
    with open(filename, "a") as f:
        f.write(f"{datetime.now()}: {result}\n")

def run_monthly():
    import time
    from datetime import datetime

    result = run_forcast()
    save_result_to_file(result)
    while True:
        now = datetime.now()
        if now.day == 1 and now.hour == 1:
            run_forcast()  # Your function
            time.sleep(86400)  # Wait a day to avoid double runs
        time.sleep(3600)

In [4]:
run_monthly()

One-Month Forecast:
            entertainment food & beverages         health        other  \
2025-01-01  Rp. 2.654.044    Rp. 3.731.897  Rp. 1.656.718  Rp. 873.248   

           transportation      utilities  
2025-01-01  Rp. 2.027.467  Rp. 2.140.453  


NameError: name 'datetime' is not defined